In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm  # for the BiT model

In [6]:
# Device configuration (Use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# Hyperparameters
batch_size = 100
num_classes = 10
learning_rate = 0.001
num_epochs = 5

# Define the transform to resize Fashion MNIST images and convert to 3 channels (RGB)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Fashion MNIST dataset
train_dataset = datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26421880/26421880 [00:07<00:00, 3688832.40it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 174648.02it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1920686.98it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5728914.03it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [8]:
# Load the BiT model (ResNet50x1)
model = timm.create_model('resnetv2_50x1_bitm', pretrained=True, num_classes=num_classes)

# Move model to the GPU device
model = model.to(device)

/home/ps/miniconda3/envs/tf/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name resnetv2_50x1_bitm to current resnetv2_50x1_bit.goog_in21k_ft_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [16]:
print(model)

ResNetV2(
  (stem): Sequential(
    (conv): StdConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0.0)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (stages): Sequential(
    (0): ResNetStage(
      (blocks): Sequential(
        (0): PreActBottleneck(
          (downsample): DownsampleConv(
            (conv): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm): Identity()
          )
          (norm1): GroupNormAct(
            32, 64, eps=1e-05, affine=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): GroupNormAct(
            32, 64, eps=1e-05, affine=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (conv2): StdConv2d(64, 64, kernel_size=(3, 3)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        # Adding tqdm to visualize progress
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100 * correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs)

Epoch 1/5: 100%|██████████| 600/600 [3:08:42<00:00, 18.87s/it]    


Epoch [1/5], Loss: 0.4505, Accuracy: 83.81%


Epoch 2/5: 100%|██████████| 600/600 [3:11:07<00:00, 19.11s/it]  


Epoch [2/5], Loss: 0.2802, Accuracy: 89.84%


Epoch 3/5: 100%|██████████| 600/600 [2:59:09<00:00, 17.92s/it]  


Epoch [3/5], Loss: 0.2342, Accuracy: 91.47%


Epoch 4/5: 100%|██████████| 600/600 [2:59:15<00:00, 17.93s/it]  


Epoch [4/5], Loss: 0.1988, Accuracy: 92.80%


Epoch 5/5: 100%|██████████| 600/600 [3:12:56<00:00, 19.29s/it]   


Epoch [5/5], Loss: 0.1778, Accuracy: 93.52%


In [13]:
from tqdm import tqdm

# Evaluation function with progress tracking
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        # Adding tqdm to visualize progress
        for images, labels in tqdm(test_loader, desc='Evaluating'):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Evaluate the model
evaluate_model(model, test_loader)

Evaluating: 100%|██████████| 100/100 [10:05<00:00,  6.06s/it]

Test Accuracy: 92.59%


In [22]:
torch.save(model.state_dict(), "teacher models/bitm_resnet50x1_FMnist.pth")